In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy.stats import norm
from pandas.tseries.offsets import *
from scipy import optimize
from tqdm import tqdm


tqdm.pandas()

In [3]:
# File position
# resource = '../../data/pulled/'
resource = '../../data/generated/'
results = '../../data/generated/'

In [23]:
def concat_dataframes(*dataframes):
    result = pd.concat(dataframes, ignore_index=True)
    return result


p1980 = pd.read_csv(resource + 'p_def_1980.csv', parse_dates=['date'])
p1981 = pd.read_csv(resource + 'p_def_1981.csv', parse_dates=['date'])
p1986 = pd.read_csv(resource + 'p_def_1986.csv', parse_dates=['date'])
p1987 = pd.read_csv(resource + 'p_def_1987.csv', parse_dates=['date'])
p1988 = pd.read_csv(resource + 'p_def_1988.csv', parse_dates=['date'])
p1989 = pd.read_csv(resource + 'p_def_1989.csv', parse_dates=['date'])
p1990 = pd.read_csv(resource + 'p_def_1990.csv', parse_dates=['date'])

results=concat_dataframes(p1980,p1981,p1986,p1987,p1988,p1989,p1990)

results['month']=results['date'].dt.month
results['year']=results['date'].dt.year

results = results.groupby(['year','month'])['p_def'].mean().reset_index()
results['p_surv'] = 1 - results['p_def']
# results['lagged_p_surv'] = results['p_surv'].shift(1)
results['dsv'] = (results['p_surv'] - results['p_surv'].shift(1)) *100

In [33]:
dsv = pd.read_excel('../../data/external/DSV.xls', header=None)
dsv = dsv.set_axis(['date', 'dsv'], axis=1)
dsv['date'] = pd.to_datetime(dsv['date'].astype(str), format='%Y%m') + MonthEnd()

In [6]:
vass_default = pd.read_csv('../../data/external/defaultdataset.txt', header=None, delimiter='\s+')
vass_default = vass_default.set_axis(['year', 'month', 'permno', 'default_probability', 'log_V_assets', 'volatility_assets'], axis=1)

perm_list = [10031,54594,29591,17057] ########################################
vass_default = vass_default[vass_default['permno'].isin(perm_list)] ##############

# vass_default = vass_default.groupby(['year','month'])['default_probability'].mean().reset_index()
# vass_default['agg_surv'] = 1 - vass_default['default_probability']
# vass_default['dsv'] = (vass_default['agg_surv'] - vass_default['agg_surv'].shift(1))*100


In [10]:
prova_default = pd.read_csv(resource + 'prova_default.csv', parse_dates=['date'])
prova_default['log_Va'] = np.log(prova_default['Va'])
prova_default = prova_default[['permno','date','log_Va','sigma_a','p_def']]

In [34]:
from scipy.stats import pearsonr

results = results[(results['year'] >= 1986) & (results['year'] <= 1990)]

results2 = dsv[(dsv['date'].dt.year >= 1986) & (dsv['date'].dt.year <= 1990)]

corr, _ = pearsonr(results['dsv'], results2['dsv'])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.685
